In [1]:
import warnings, os
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from glob import glob


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from aggmap import AggMap, loadmap
import seaborn as sns


import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)
    ber =  (fp / (fp + tn) + fn / (tp + fn)) * 0.5

    print('acc: %.3f, roc-auc: %.3f, BER: %.3f' % (acc,auc,ber))

    return acc, auc


def get_best_params(X, y):
    clf = lgb.LGBMClassifier(verbose=-1, force_col_wise=True)
    parameters = {'n_estimators': [ 50, 100, 200],
                  'max_depth': [4,  6, 10]}

    grid = GridSearchCV(clf, parameters, scoring='roc_auc', n_jobs=9, iid=False, cv=5, verbose=0)
    gres = grid.fit(X, y)
    return grid.best_params_

In [2]:
mp = loadmap('/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S-G.mp')
lst = glob('/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/*.h5')

n_fold = 5

res = []
for p in lst:

    print('#'*50 + ' %s ' % p + '#'*50 )
    dfx = pd.read_hdf(p, key = 'expression')
    dfy = pd.read_hdf(p, key = 'labels')
    print(p, dfy.value_counts().to_dict())

    outer = KFold(n_splits = n_fold, shuffle = True, random_state = 123)
    outer_idx = list(outer.split(dfx, dfy))
    
    fold_all = []
    for i, idx in enumerate(outer_idx):
        fold_num = "fold_%s" % str(i).zfill(2) 
        train_idx, test_idx = idx

        # subset feature selection by the fold change
        dfxy_train = dfx.iloc[train_idx].join(dfy.iloc[train_idx])
        abds = dfxy_train.groupby(dfy.name).mean()
        fc = abs(abds.iloc[0] - abds.iloc[1])
        selected = fc[fc > 0.5].index

        trainX = dfx[selected].iloc[train_idx].values
        trainY = dfy.iloc[train_idx].values

        testX = dfx[selected].iloc[test_idx].values
        testY = dfy.iloc[test_idx].values
        
        ## training
        print("\nInput train and test X shape is %s, %s  \n" % (trainX.shape,  testX.shape))
        print("Getting the best parameters by gridsearch \n")
        best_params = get_best_params(trainX, trainY)
        clf = lgb.LGBMClassifier(**best_params, verbose=-1, n_jobs=-1, force_col_wise=True)
        clf.fit(trainX, trainY)

        ## evaluation
        y_true = testY
        y_score = clf.predict_proba(testX)[:,1]
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        acc, auc = score(dfr)
        
        fold_all.append(auc)
        res.append([auc, i, p, len(selected), best_params])
        
    print('The auc score for %s is %s.' % (p, np.mean(fold_all)))


################################################## /home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5 ##################################################
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5 {0: 250, 1: 124}

Input train and test X shape is (299, 2260), (75, 2260)  

Getting the best parameters by gridsearch 

acc: 0.760, roc-auc: 0.732, BER: 0.319

Input train and test X shape is (299, 2854), (75, 2854)  

Getting the best parameters by gridsearch 

acc: 0.640, roc-auc: 0.561, BER: 0.483

Input train and test X shape is (299, 1785), (75, 1785)  

Getting the best parameters by gridsearch 

acc: 0.653, roc-auc: 0.722, BER: 0.420

Input train and test X shape is (299, 1909), (75, 1909)  

Getting the best parameters by gridsearch 

acc: 0.653, roc-auc: 0.729, BER: 0.416

Input train and test X shape is (300, 1627), (74, 1627)  

Getting the best parameters by gridsearch 

acc: 0.608, roc-auc: 0.752, BER: 0.490


In [3]:
dfres = pd.DataFrame(res, columns = ['score', 'fold', 'task', 'selected_features','best_params'])
dfres.to_csv('./FS_LGB_5FCV_results_grade.csv')
dfres.groupby('task').mean().sort_values('score',ascending=False)

,score,fold,selected_features
task,,,
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_UCEC_grade.h5,0.889864,2.0,5395.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_STAD_grade.h5,0.764846,2.0,3569.6
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LGG_grade.h5,0.736323,2.0,2952.6
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_test_HNSC_grade.h5,0.728773,2.0,1688.8
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5,0.699148,2.0,2087.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_PAAD_grade.h5,0.628194,2.0,2188.8
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_CESC_grade.h5,0.603956,2.0,1971.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_KIRC_grade.h5,0.583777,2.0,400.0


In [4]:
dfr = dfres.groupby('task').mean().sort_values('score',ascending=False)
order = ['CESC','KIRC','LGG','LIHC','PAAD','STAD','UCEC','HNSC']

dfr.index = dfr.index.map(lambda x:x.split('_')[-2])
dfr = dfr.loc[order]

In [5]:
dfr

,score,fold,selected_features
task,,,
CESC,0.603956,2.0,1971.0
KIRC,0.583777,2.0,400.0
LGG,0.736323,2.0,2952.6
LIHC,0.699148,2.0,2087.0
PAAD,0.628194,2.0,2188.8
STAD,0.764846,2.0,3569.6
UCEC,0.889864,2.0,5395.0
HNSC,0.728773,2.0,1688.8
